In [20]:
#!/usr/bin/env python
# coding: utf-8

# -------------------------------------------------------------------------------------------------------------
#   python3 requete-v1.py
#
#   Traitement d'une requête :
#       0. lecture fichier des index (inversé & documents)
#       1. lecture de la requête
#       2. extraction de ses tokens 
#       - lemmatisation
#       - normalisation 
#       3. recherche des documents correspondants dans l'index
#           par comptage des matchs
#       4. affichage des résultats
# 
#   remarque :  ne donne pas d'info sur les termes trouvés
# -------------------------------------------------------------------------------------------------------------

import os
import sys
import re
import pickle 
import json
from fonctions_index import litTexteDuFichier, ecritTexteDansUnFichier, lemmatiseTexte, normaliseTokens, normaliseTokensRequete, lemmatiseTokenRequete

PARSEUR = 'TALISMANE'
   
fiIndex = "./_index/indexInverse"
fiDocs = "./_index/indexDocuments"
doCorpus = "../corpus/miniCorpus/FR/"
corpus_sansBalise = "../corpus/sans_balises"

fiTxt = "./_log/tempo.txt" 

fiLog = "./_log/requete.log"
log = ""


indexDocuments = dict()
indexInverse = dict ()

# 0. lecture des index
if os.path.isfile (fiDocs) :
    with open(fiDocs, 'rb') as FI :
        #indexDocuments = pickle.load(FI)
        indexDocuments = json.load(FI)
    print ("index de cuduments bien lu")

if os.path.isfile (fiIndex) :
    with open(fiIndex, 'rb') as FI :
        #indexInverse = pickle.load(FI)
        indexInverse = json.load(FI)
    print ("index inversé bien lu")
 
# 1. lecture de la requête (mots clés)
print("votre requête ?")
requete = input()
print("...", requete)
log += "\nrequête: %s \n" % (requete)

# 2. extraction de ses tokens (lemmatisation, normalisation)
ecritTexteDansUnFichier (requete, fiTxt)   
tokens = lemmatiseTokenRequete (fiTxt)
termes = normaliseTokensRequete (tokens)
log += "\ntermes: %s \n" % (termes)

# 3. recherche des termes dans l'index
nbMatch = dict ()
for terme in termes:
    # pour chaque terme de la requête
    if terme in indexInverse:
        for liste in indexInverse[terme]:
            if nbMatch.__contains__(liste[0]):
                nbMatch[liste[0]][terme] = liste[1]
            else:
                valeur = {terme:liste[1]}
                nbMatch[liste[0]] = valeur

log += "\nrésultat: %s \n" % (str (nbMatch) )
log += "\nresultats détaillées: \n"

# 4. affichage des résultats
print(f"\n{nbMatch}")
print("{nombre} documents ont été trouvés!\n".format(nombre=len(nbMatch)))

for doc, nb in nbMatch.items():#sorted(nbMatch.items(), key=lambda item: item[0], reverse=True) :
    info_document = indexDocuments[str(doc)]
    #print(f"{info_document} --> {nb}")

    #stocker les infos dans log
    nom_fichier = info_document[0]
    titre_complet = info_document[1]
    if "\n\n" in titre_complet:
        titre_principal = titre_complet[:titre_complet.index("\n\n")]
        sous_titre = titre_complet[titre_complet.index("\n\n")+2:]
        log += f"nom de fichier: {nom_fichier}\n titre de texte: {titre_principal}\n sous-titre: {sous_titre}\n termes trouvés dans ce fichier: {nb}\n\n"
    else:
        log += f"nom de fichier: {nom_fichier}\n titre de texte: {titre_complet}\n termes trouvés dans ce fichier: {nb}\n\n"

#5 calculer les scores avec TF-IDF






# sauvegarde du log
ecritTexteDansUnFichier (log, fiLog)

index de cuduments bien lu
index inversé bien lu
votre requête ?
chomage citoyen culture day department
... chomage citoyen culture day department

{14: {'chomage': 3, 'citoyen': 1, 'culture': 5}, 22: {'chomage': 1, 'citoyen': 1}, 0: {'citoyen': 1}, 5: {'citoyen': 2}, 10: {'citoyen': 3}, 1: {'culture': 1}, 6: {'culture': 1}, 8: {'culture': 2}, 21: {'culture': 2, 'day': 4}, 24: {'culture': 2, 'day': 3, 'department': 5}, 26: {'culture': 1}, 28: {'culture': 5, 'day': 1}, 3: {'day': 1, 'department': 1}, 13: {'day': 1}, 17: {'day': 2}, 19: {'day': 5, 'department': 2}, 29: {'day': 2}, 12: {'department': 1}}
18 documents ont été trouvés!



In [43]:
mots_list = "je -ne +sais -pas"
tokens = mots_list.split(" ")
tokens_sans_ponc=[]

print(tokens)
for i in tokens:
    if i[0] == "-" or i[0] == "+":
        tokens_sans_ponc.append(i[1:])
    else:
        tokens_sans_ponc.append(i)
cave = " ".join(tokens_sans_ponc)
cave

['je', '-ne', '+sais', '-pas']


'je ne sais pas'

In [38]:
aa = ""
aa.join(["s","o","c","i","a",'l'],split=" ")

TypeError: join() takes no keyword arguments

In [ ]:
#删除加减号的版本
def lemmatiseTokensRequete(fichier):
    tokensLemmePos = []
    fiParse = fichier + ".par"

    #supprimer le "+" et le "-" des termes de la requête
    requete_termes = litTexteDuFichier(fichier)
    termes_avec_ponc = requete_termes.split(" ")
    liste_termes_sans_ponc = []

    for terme in termes_avec_ponc:
        if terme[0] == "-" or terme[0] == "+":
            liste_termes_sans_ponc.append(terme[1:])
        else:
            liste_termes_sans_ponc.append(terme)
    termes_a_traiter = " ".join(liste_termes_sans_ponc)

    #écrire à nouveau les termes sans ponctuations dans le fichier "tempo.txt"
    ecritTexteDansUnFichier(termes_a_traiter, fichier)
    
    # execute TreeTagger
    if detect(requete_termes) == "fr":
        cmd = "tree-tagger-french"
    else:
        cmd = "tree-tagger-english"
    os.system("%s %s > %s" % (cmd, fichier, fiParse))

    # lit le csv produit par treeTagger
    with open(fiParse, "r") as FI:
        for ligne in FI:
            ligne = ligne.strip()
            defToken = ligne.split("\t")
            # filtre les tokens vides
            if len(defToken) >= 3:
                if defToken[2] != "<unknown>":
                    # ajouter les tokens lemmatisés dans la liste 
                    tokensLemmePos.append(defToken[2])
                else:
                    tokensLemmePos.append(defToken[0])
    # renvoie une liste de tokens (lemme)
    return tokensLemmePos

In [ ]:
# copie de la fonction de "fonctions_index.py"
def lemmatiseTokenRequete(fichier):
    tokensLemmePos = []
    fiParse = fichier + ".par"


    # execute TreeTagger
    cmd = "tree-tagger-french"
    os.system("%s %s > %s" % (cmd, fichier, fiParse))
    # lit le csv produit par treeTagger
    with open(fiParse, "r") as FI:
        for ligne in FI:
            ligne = ligne.strip()
            defToken = ligne.split("\t")
            # filtre les tokens vides
            if len(defToken) >= 3:
                if defToken[2] != "<unknown>":
                    # ajouter les tokens lemmatisés dans la liste 
                    tokensLemmePos.append(defToken[2])
                else:
                    tokensLemmePos.append(defToken[0])
    # renvoie une liste de tokens (lemme)
    return tokensLemmePos

In [49]:
resultats = {14: {'chomage': 3, 'citoyen': 1, 'economie': 3, 'societe': 1}, 22: {'chomage': 1, 'citoyen': 1, 'economie': 2, 'societe': 1}, 0: {'citoyen': 1, 'economie': 1}, 5: {'citoyen': 2, 'societe': 1}, 10: {'citoyen': 3, 'societe': 5}, 4: {'economie': 4, 'societe': 3, 'salaire': 3}, 7: {'economie': 1, 'societe': 1}, 11: {'economie': 4, 'societe': 1}, 15: {'economie': 1, 'societe': 4}, 26: {'economie': 9, 'societe': 1}} 

In [50]:
resultats

{14: {'chomage': 3, 'citoyen': 1, 'economie': 3, 'societe': 1},
 22: {'chomage': 1, 'citoyen': 1, 'economie': 2, 'societe': 1},
 0: {'citoyen': 1, 'economie': 1},
 5: {'citoyen': 2, 'societe': 1},
 10: {'citoyen': 3, 'societe': 5},
 4: {'economie': 4, 'societe': 3, 'salaire': 3},
 7: {'economie': 1, 'societe': 1},
 11: {'economie': 4, 'societe': 1},
 15: {'economie': 1, 'societe': 4},
 26: {'economie': 9, 'societe': 1}}

In [151]:
import copy
resultats_finals = copy.deepcopy(resultats)


In [152]:
resultats_finals

{14: {'chomage': 3, 'citoyen': 1, 'economie': 3, 'societe': 1},
 22: {'chomage': 1, 'citoyen': 1, 'economie': 2, 'societe': 1},
 0: {'citoyen': 1, 'economie': 1},
 5: {'citoyen': 2, 'societe': 1},
 10: {'citoyen': 3, 'societe': 5},
 4: {'economie': 4, 'societe': 3, 'salaire': 3},
 7: {'economie': 1, 'societe': 1},
 11: {'economie': 4, 'societe': 1},
 15: {'economie': 1, 'societe': 4},
 26: {'economie': 9, 'societe': 1}}

In [153]:
requete = ["Chômages","citoyens","+Économie","+sociétés","-salaire"]

In [154]:
requete_finale = ["chomage","citoyen","+economie","+societe","-salaire"]#7,11,15,26

In [163]:
rrr = {}
for document in resultats_finals:
    for terme in requete_finale:
        if terme.startswith("-") and terme[1:] in resultats_finals[document]:
            resultats_finals.pop(document)
        else:
            pass

KeyError: 10

In [197]:
ag = list(resultats_finals.items())
mots_a_traiter = ag[0][1]

In [198]:
mots_a_traiter

{'economie': 4, 'societe': 3, 'salaire': 3}

In [201]:
all(term in mots_a_traiter for term in requete_finale)

False

In [207]:
a = (term in mots_a_traiter for term in requete_finale)

In [209]:
list(a)

[False, False, False, False, False]

In [213]:
a = list(term in mots_a_traiter for term in requete_finale)
a

[False, False, False, False, False]

In [214]:
for terme in requete_finale:
    for document in resultats_finals:
        if (terme[0] == "+") and (terme[1:] in resultats_finals[document]):
            if document in rrr:
                pass
            else:
                rrr[document] = resultats_finals.get(document)
        

In [219]:
inputt = input()
aa = list(inputt.split(" "))
print(aa)

dfs +dsdfd -sfe +dd
['dfs', '+dsdfd', '-sfe', '+dd']


In [240]:
import treetaggerwrapper

In [241]:
tokensLemmePos = []
    texte = extraitTexteDuFichier(fichier, nom_fichier)
    if detect(texte) == "fr":
        tagger = treetaggerwrapper.TreeTagger(TAGLANG="fr")
        tags = tagger.tag_text(texte)
        lang = "FR"
    else:
        tagger = treetaggerwrapper.TreeTagger(TAGLANG="en")
        tags = tagger.tag_text(texte)
        lang = "EN"
    for ligne in tags:
        ligne = ligne.strip()
        defToken = ligne.split("\t")
        # filtre les tokens vides
        if len(defToken) >= 3:
            # reduit aux lemmes et pos
            token = [defToken[2], defToken[1]]
            tokensLemmePos.append(token)
    # renvoie une liste de tokens (lemme et pos)
    return tokensLemmePos, lang

IndentationError: unexpected indent (<ipython-input-241-ae39d607adce>, line 2)

In [246]:
termes_test = ["chômages","salaires","citoyenne"]

In [248]:
tagger = treetaggerwrapper.TreeTagger(TAGLANG="fr")
aa = tagger.tag_text(termes_test)

In [249]:
aa

['chômages\tNOM\tchômage', 'salaires\tNOM\tsalaire', 'citoyenne\tNOM\tcitoyen']

In [256]:
for i in aa:
    i = i.strip()
    token = i.split("\t")
    

In [259]:
text = " ".join(termes_test)
text

'chômages salaires citoyenne'

In [254]:
from langdetect import detect

In [260]:
detect(text)

'fr'

In [261]:
tagger = treetaggerwrapper.TreeTagger(TAGLANG="fr")
aa = tagger.tag_text(text)

In [262]:
aa

['chômages\tNOM\tchômage', 'salaires\tNOM\tsalaire', 'citoyenne\tNOM\tcitoyen']

In [263]:
def lemmatiseTermes(liste):
    tokensLemme = []
    # execute TreeTagger
    texte = " ".join(liste)
    if detect(texte) == "fr":
        tagger = treetaggerwrapper.TreeTagger(TAGLANG="fr")
        tags = tagger.tag_text(texte)
    else:
        tagger = treetaggerwrapper.TreeTagger(TAGLANG="en")
        tags = tagger.tag_text(texte)
        lang = "EN"
    for ligne in tags:
        ligne = ligne.strip()
        defToken = ligne.split("\t")
        # filtre les tokens vides
        if len(defToken) >= 3:
            # reduit aux lemmes et pos
            token = [defToken[2]]
            tokensLemme.append(token)
    # renvoie une liste de tokens (lemme et pos)
    return tokensLemme

In [264]:
lemmatiseTermes(termes_test)

[['chômage'], ['salaire'], ['citoyen']]

In [ ]:

def lemmatiseTokensRequete(fichier):
    tokensLemmePos = []
    fiParse = fichier + ".par"
    
    # execute TreeTagger
    termes_requete = litTexteDuFichier(fichier)
    if detect(termes_requete) == "fr":
        cmd = "tree-tagger-french"
    else:
        cmd = "tree-tagger-english"
    os.system("%s %s > %s" % (cmd, fichier, fiParse))

    # lit le csv produit par treeTagger
    with open(fiParse, "r") as FI:
        for ligne in FI:
            ligne = ligne.strip()
            defToken = ligne.split("\t")
            # filtre les tokens vides
            if len(defToken) >= 3:
                if defToken[2] != "<unknown>":
                    # ajouter les tokens lemmatisés dans la liste 
                    tokensLemmePos.append(defToken[2])
                else:
                    tokensLemmePos.append(defToken[0])
    # renvoie une liste de tokens (lemme)
    return tokensLemmePos